In [30]:
import datetime as dt
import sqlalchemy as sa
import os
import sys
import pandas as pd

In [31]:
username = os.getlogin()
external_folder_path = 'C:/Users/' + username + '/Documents/Projects/Financial_Securities/Custom_Python_Functions/'
sys.path.append(external_folder_path)
from custom_python_functions import create_connection, clear_table, load_key, decrypt

key1 = 'user_key.ky'
key_file1 = 'user_key.txt'
key2 = 'pass_key.ky'
key_file2 = 'pass_key.txt'

key1 = load_key(external_folder_path, key1)
uid = decrypt(external_folder_path, key_file1, key1)

key2 = load_key(external_folder_path, key2)
passwd = decrypt(external_folder_path, key_file2, key2)

# Setup connection parameters
server = 'danvuk.database.windows.net'
dbase = 'Financial_Securities'

# Create a connection to the database
s, e = create_connection(server, dbase, uid, passwd)
s1 = s()  # Instantiate a session object

In [28]:
Base = sa.orm.declarative_base()

class Data_STG(Base):
    
    """
    SQLAlchemy ORM class representing the 'Data_STG' table in the 'Equities' schema.

    Attributes:
        __tablename__ (str): The name of the table in the database.
        __table_args__ (dict): Additional arguments for the table, including schema name.
        Date (Column): The date column, part of the composite primary key.
        Description (Column): A description column, part of the composite primary key.
        Int_Value1 (Column): An integer column.
        Int_Value2 (Column): Another integer column.
    """
    
    __tablename__ = 'Data_STG'
    __table_args__ = {'schema': 'Equities'}
    Date = sa.Column(sa.Date, primary_key=True)
    Description = sa.Column(sa.String, primary_key=True)
    Int_Value1 = sa.Column(sa.Integer)
    Int_Value2 = sa.Column(sa.Integer)

In [32]:

# Clear the existing data in the Data_STG table
clear_table(s1, 'Financial_Securities.Equities.Data_STG')

In [33]:
in_file = 'C:/Users/' + username + '/Documents/Projects/Financial_Securities/Data_Files/GICS_Industries.csv'  # Path to the CSV file

curr_date = dt.datetime.now()  # Current date and time


if os.path.isfile(in_file):
    # Load the CSV file into a DataFrame
    df_industry_groups = pd.read_csv(in_file)

    # Select relevant columns
    df_industry_groups = df_industry_groups[['Industry_Group_ID', 'Industry_Group', 'Sector_ID']].drop_duplicates()
    
    # Iterate through the rows of the DataFrame
    for index, row in df_industry_groups.iterrows():
        try:
            # Create a new Data_STG instance for each row
            q1 = Data_STG(
                Date=curr_date,
                Description=row['Industry_Group'],
                Int_Value1=row['Industry_Group_ID'],
                Int_Value2=row['Sector_ID']
            )
            s1.add(q1)  # Add the instance to the session
            
        # Handle SQLAlchemy errors if they occur during adding the object
        except sa.exc.SQLAlchemyError as e:
            # Print the error    
            message = f"Issue with updating Data_STG database table for Date: {curr_date} and Industry_Group: {row['Industry_Group']}. Error: {e}"
            print(message)
    
    # Commit all changes to the database
    s1.commit()
    
    print("Database data load is complete")
    
    # SQL query to count the number of records in the Sectors table
    sql_stat = """SELECT COUNT(*) FROM [Financial_Securities].[Equities].[Data_STG]"""

    try: 
        result = e.execute(sql_stat)  # Execute the count query
        cnt_recs = result.scalar()  # Get the count of records
    
     # Handle SQLAlchemy errors if they occur during query execution
    except sa.exc.SQLAlchemyError as e:
        # Print the error
        print(f"Issue querying Data_STG database table for count! Error: {e}")

        
    # Compare the record counts and print the result    
    if cnt_recs < len(df_industry_groups):
        print(f"Only {cnt_recs} records out of {len(df_industry_groups)} records were loaded into the Data_STG table!")
    else:
        print(f"All {cnt_recs} records were loaded into the Data_STG table!")
else:
    print("File not found!")  # Print a message if the file does not exist
        

All 25 records were loaded into the Data_STG table!


In [34]:
s1.close()  # Close the session